<a href="https://colab.research.google.com/github/yudhiesh/PyTorch/blob/master/PersonalityPrediction_myPersonality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Feb 21 18:47:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    22W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers==3
!pip install pytorch-lightning

In [3]:
import torch 
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset,DataLoader
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix, classification_report
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional.classification import auroc
import re
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

In [4]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [5]:
PATH = "/content/drive/MyDrive/BLI_Data/data.csv"
df = pd.read_csv(PATH)

In [6]:
df.sample(10)

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
5274,d39c2b0fb2e50e37795fdbe3b8cd3792,.......so apathetic........,3.75,2.00,5.00,4.25,4.50,y,n,y,y,y,10/13/09 07:16 PM,775.0,290973.00,97.27,0.01,296174.0,0.50,0.08
7853,dba59aed04c6759a955a37f021e45bb7,doing a double shift at work.,1.75,3.25,2.40,3.05,3.05,n,y,n,n,n,09/11/09 04:23 AM,91.0,3763.98,93.98,0.05,3899.0,0.49,0.09
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3.00,3.15,3.25,4.40,n,y,n,n,y,06/19/09 03:21 PM,180.0,14861.60,93.29,0.03,15661.0,0.49,0.10
8816,b84d2613c4cf4e73f9c230f57facf66c,had to laugh when the doctor was cutting off h...,3.50,3.25,3.25,3.00,3.75,n,y,n,n,n,01/31/10 09:54 AM,243.0,27562.00,94.52,0.03,28498.0,0.49,0.15
3149,b4a21c82de4011033c8ac67081ff939c,I think that *PROPNAME* was embarassed that we...,4.55,2.20,3.30,3.90,3.65,y,n,n,y,n,11/09/09 02:44 AM,254.0,30611.80,96.03,0.02,31359.0,0.49,0.09
7088,540159466df2f050426692ddaac1eab4,doesn't like it when things don't go according...,3.00,1.75,4.00,4.75,4.00,n,n,y,y,y,12/15/09 01:39 AM,315.0,47078.60,95.80,0.03,48149.0,0.49,0.19
4207,dba5f5266d03dd6d4db084ad7dbc683c,just tried kimchi and suchi! :D,1.55,2.45,3.50,3.40,4.50,n,n,n,n,y,10/11/09 05:03 AM,265.0,33752.40,97.22,0.02,34425.0,0.50,0.03
6381,b576dc0449e75306c30456902ce0c80b,Random girl (In a bus) : Are you half chinese?...,3.84,2.60,3.75,4.00,4.15,y,n,y,y,y,09/25/09 07:09 AM,254.0,29724.90,93.25,0.04,30978.0,0.49,0.18
7845,5880081cd3de1619cd431a75d9052dfc,Why is it that everyone else's musical tastes ...,2.90,2.20,3.05,3.90,4.10,n,n,n,y,y,10/06/09 02:08 AM,72.0,2015.24,81.10,0.12,2262.0,0.46,0.38
8666,7f3bfec0b7228d0900b01fbc8ce9d59f,"""... it's a very dangerous time, the coalition...",2.25,3.50,3.25,2.75,2.25,n,y,n,n,n,10/22/09 03:29 PM,1098.0,593081.00,98.66,0.01,598066.0,0.50,0.04


In [7]:
# Range of values where for each class being either positive or negative
(pd.wide_to_long(df.reset_index(), i='index',
                stubnames=['c','s'], j='cat',
                suffix='.*' )
   .groupby(['cat','c'])['s'].agg(['min','max'])
)


min   max
cat c            
AGR n  1.65  3.50
    y  3.55  5.00
CON n  1.45  3.45
    y  3.50  5.00
EXT n  1.33  3.55
    y  3.60  5.00
NEU n  1.25  2.75
    y  2.80  4.75
OPN n  2.25  3.75
    y  3.80  5.00

In [8]:
cols = ["cEXT","cNEU","cAGR","cCON","cOPN"]

df[cols] = df[cols].replace({"n" : 0, "y" : 1})

In [9]:
cols_ = ["STATUS","cEXT","cNEU","cAGR","cCON","cOPN"]
df = df[cols_]

In [10]:
df.head()

,STATUS,cEXT,cNEU,cAGR,cCON,cOPN
0,likes the sound of thunder.,0,1,0,0,1
1,is so sleepy it's not even funny that's she ca...,0,1,0,0,1
2,is sore and wants the knot of muscles at the b...,0,1,0,0,1
3,likes how the day sounds in this new song.,0,1,0,0,1
4,is home. <3,0,1,0,0,1


In [ ]:
def sub_preprocess(sub):
  # run regex to remove certain characters
  sub['STATUS'] = sub['STATUS'].map(lambda x: re.sub(r"[@\?\.$%_\[\]()+-:*\"]", ' ', x, flags=re.I))
  sub['STATUS'] = sub['STATUS'].map(lambda x: re.sub(r"[,']", '', x, flags=re.I))
  sub['STATUS'] = sub['STATUS'].map(lambda x: re.sub("(?<![\w'])\w+?(?=\b|'s)", ' ', x))# run regex to remove line breaks and tabs
  sub['STATUS'] = sub['STATUS'].map(lambda x: re.sub(r"\s+", ' ', x))

sub_preprocess(df)

In [ ]:
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

df['STATUS'] = df['STATUS'].apply(lambda x: clean_text_round1(x))

In [ ]:
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

df['STATUS'] = df['STATUS'].apply(lambda x: clean_text_round2(x))

In [ ]:
df.sample(10)

,STATUS,cEXT,cNEU,cAGR,cCON,cOPN
8511,likes food.,1,0,1,1,1
2895,better ace my exam considering i missed the be...,1,0,1,1,1
3196,Dear Baby Jesus: Please compell my boyfriend t...,1,0,0,1,0
7986,"Ladies and gentlemen, we are floating in space...",0,1,0,0,0
9758,"Go Gators! 8 and 0.... Now, lets see if the Vo...",0,1,1,0,1
6352,My new froggies are so cute....names are *PROP...,1,0,0,0,0
1368,~ grr... school sucks.,0,1,0,1,1
692,saw the official end of the New England Era. ...,0,0,0,0,1
3095,is not with his darling anymore :( sad day(s)....,1,0,1,1,1
9590,didn't get home until 5:30am ... cause she had...,1,1,0,1,1


In [11]:
df['lists'] = df[df.columns[1:6]].values.tolist()

In [12]:
df = df[['STATUS','lists']]

In [13]:
df.sample(10)

,STATUS,lists
3196,Dear Baby Jesus: Please compell my boyfriend t...,"[1, 0, 0, 1, 0]"
5684,is missing you.,"[0, 0, 1, 0, 1]"
649,and the line between yin and yang.,"[0, 0, 0, 0, 1]"
1162,is excited for the newest movie adaptation of ...,"[0, 0, 1, 0, 1]"
8587,Santa isn't bringing me presents this year. He...,"[0, 0, 0, 0, 0]"
2617,is always surprised at how many building mater...,"[0, 0, 1, 1, 1]"
9551,coastin......;-),"[1, 0, 1, 1, 1]"
1113,got sick of the crap �5 haircuts so upgraded t...,"[0, 0, 0, 0, 0]"
5299,thinks she may have to walk to class in the mo...,"[1, 0, 1, 1, 1]"
4660,"work, work, work until 6. Day-off tomorrow to ...","[1, 0, 1, 1, 1]"


In [25]:
MAX_LEN = 200
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 20
LEARNING_RATE = 1e-5
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [27]:
import random

seed_val = 42 #so our results/process is reproducible by whoever wants to reproduce
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val) #include for when using a GPU

In [28]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.STATUS = dataframe.STATUS
        self.targets = self.data.lists
        self.max_len = max_len

    def __len__(self):
        return len(self.STATUS)

    def __getitem__(self, index):
        STATUS = str(self.STATUS[index])
        STATUS = " ".join(STATUS.split())

        inputs = self.tokenizer.encode_plus(
            STATUS,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [29]:
train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=seed_val)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (9917, 2)
TRAIN Dataset: (7934, 2)
TEST Dataset: (1983, 2)


In [30]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 1
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [37]:
from transformers import BertTokenizer, BertModel, BertConfig

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 5)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [38]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [39]:
def train_fn(data_loader, model, optimizer, device, scheduler, epoch):
  # Put the model in training mode
  model.train()

  # loop over all the batches
  for i, d in enumerate(data_loader):
    ids = d["ids"]
    token_type_ids = d["token_type_ids"]
    mask = d["mask"]
    targets = d["targets"]

    # convert to device type
    ids = ids.to(device, dtype=torch.long)
    token_type_ids = token_type_ids.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)
    targets = targets.to(device, dtype=torch.float)

    # zero-grad the optimizer
    optimizer.zero_grad()
    outputs = model(
        ids=ids,
        mask=mask,
        token_type_ids=token_type_ids
    )
    # calculate the loss
    loss = loss_fn(outputs, targets)
    if i % 5000==0:
      print(f'Epoch: {epoch}, Loss:  {loss.item()}')
    # backward step the loss
    loss.backward()
    # step the optimizer
    optimizer.step()
    # step scheduler
    scheduler.step()


In [40]:
def eval_fn(data_loader, model, device):
  # put the model in eval mode
  model.eval()
  # initialize empty lists for targets and outputs
  fin_targets = []
  fin_outputs = []
  
  # no_grad to prevent unnecessary use of GPU memory
  with torch.no_grad():
    for d in data_loader:
      ids = d["ids"]
      token_type_ids = d["token_type_ids"]
      mask = d["mask"]
      targets = d["targets"]
  
      # convert to device type
      ids = ids.to(device, dtype=torch.long)
      token_type_ids = token_type_ids.to(device, dtype=torch.long)
      mask = mask.to(device, dtype=torch.long)
      targets = targets.to(device, dtype=torch.float)
  
      outputs = model(
          ids=ids,
          mask=mask,
          token_type_ids=token_type_ids
      )
  
      # convert targets to cpu and extend the final list
      targets = targets.cpu().detach()
      fin_targets.extend(targets.numpy().tolist())
      outputs = torch.sigmoid(outputs).cpu().detach()
      fin_outputs.extend(outputs.numpy().tolist())
  return fin_outputs, fin_targets


In [41]:
optimizer = AdamW(
    model.parameters(),
    lr=LEARNING_RATE, #2e-5 > 5e-5: A HYPERPARAMETER
    eps=1e-8
)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(training_loader) * EPOCHS
)

In [42]:
best_f1_score = 0
for epoch in range(EPOCHS):
  train_fn(training_loader,model, optimizer, device, scheduler, epoch)
  outputs, targets = eval_fn(testing_loader, model, device)
  outputs = np.array(outputs) >= 0.5
  accuracy = accuracy_score(targets,outputs)
  print(f"Accuracy score = {accuracy}")
  f1_score_micro = f1_score(targets, outputs, average='micro')
  f1_score_macro = f1_score(targets, outputs, average='macro')
  confusion_matrix = multilabel_confusion_matrix(targets, outputs)
  print(f"Accuracy Score = {accuracy}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")
  print(classification_report(targets, outputs, zero_division=0))
  if f1_score_micro > best_f1_score:
    torch.save(model.state_dict(), f"/content/drive/MyDrive/BLI_Data/model-epoch-{epoch}.bin")
    best_f1_score = f1_score_micro

Epoch: 0, Loss:  0.7076229453086853
Accuracy score = 0.10993444276348967
Accuracy Score = 0.10993444276348967
F1 Score (Micro) = 0.6192307692307693
F1 Score (Macro) = 0.4967605681463379
              precision    recall  f1-score   support

           0       0.51      0.39      0.44       832
           1       0.00      0.00      0.00       733
           2       0.59      0.67      0.63      1049
           3       0.51      0.63      0.56       880
           4       0.75      1.00      0.85      1478

   micro avg       0.62      0.62      0.62      4972
   macro avg       0.47      0.54      0.50      4972
weighted avg       0.52      0.62      0.56      4972
 samples avg       0.66      0.60      0.57      4972

Epoch: 1, Loss:  0.6534436345100403
Accuracy score = 0.1356530509329299
Accuracy Score = 0.1356530509329299
F1 Score (Micro) = 0.6271257610749528
F1 Score (Macro) = 0.5368998573204807
              precision    recall  f1-score   support

           0       0.56      0.3